In [1]:
import sys
sys.path.append('../')
sys.path.append('../../')
 
from db_data_downloader.downloader_base import DownloaderBase
from db_factor_prebuilder.utils.expression_excutor import AlphaExpressionExcutor
import json
import sqlite3
import database_config as db_config

db_conn = sqlite3.connect('../../hh_quant_auto.db')
db_downloader = DownloaderBase(db_conn, db_config)
exp_excutor = AlphaExpressionExcutor()

alpha_factor_dict = json.loads(open("../factor_lib/alpha_101.json", "r").read())
stock_base = db_downloader._download_history_base_info('sh.600011', '2013-01-01', '2014-01-01')

In [2]:
dataframe = stock_base[["code", "datetime"]]
for alpha_name, alpha_expression in alpha_factor_dict.items():
    print(f"{alpha_name}: {alpha_expression}")
    dataframe[alpha_name] = exp_excutor.excute(stock_base, alpha_expression)

alpha_001: (rank(tsargmax(signedpower(where(((close/shift(close,1)-1) < 0), std((close/shift(close,1)-1), 20), close), 2), 5)) -0.5)
alpha_002: (-1 * corr(rank(delta(log(volume), 2)), rank(div((close - open), open)), 6))
alpha_003: (-1 * corr(rank(open), rank(volume), 10))
alpha_004: (-1 * tsrank(rank(low), 9))
alpha_005: (rank((open - (sum(((high+low+open+close)*0.25), 10) / 10))) * (-1 * abs(rank((close - ((high+low+open+close)*0.25))))))
alpha_006: (-1 * corr(open, volume, 10))
alpha_007: where((mean(volume,20) < volume), ((-1 * tsrank(abs(delta(close, 7)), 60)) * sign(delta(close, 7))), (-1* 1))
alpha_008: (-1 * rank(((sum(open, 5) * sum((close/shift(close,1)-1), 5)) - shift((sum(open, 5) * sum((close/shift(close,1)-1), 5)),10))))
alpha_009: where((0 < tsmin(delta(close, 1), 5)), delta(close, 1), where((tsmax(delta(close, 1), 5) < 0), delta(close, 1), (-1 * delta(close, 1))))
alpha_010: rank(where((0 < tsmin(delta(close, 1), 4)), delta(close, 1), where((tsmax(delta(close, 1), 4) < 

NameError: name 'market_cap_0' is not defined